<a href="https://colab.research.google.com/github/MayaAllam/BigMart-Data-Analysis-and-Prediction/blob/main/Internshipcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from xgboost import XGBRegressor
warnings.filterwarnings("ignore")
upload=files.upload()
upload=files.upload()

In [ ]:
# Problem Statment:The data has missing values as some stores do not report all the data due to technical glitches. Hence, it will be required to treat them accordingly.
# Hypothesis Generation:Item type,Item MRP, Stores established


In [ ]:
data_train = pd.read_csv("Train.csv")
data_test = pd.read_csv("Test.csv")

In [ ]:
data_train.head()


In [ ]:
data_test.head()

In [ ]:
print('Train Data',data_train.shape)
print('Test Data',data_test.shape)
data_train.info()
data_train.boxplot()

In [ ]:
print('No of Training examples:', len(data_train),'\n')
data_train.describe().T.style.background_gradient(cmap='BuPu')

In [ ]:
numericalfeatures = data_train.select_dtypes(include = ['int64', 'Int64','float64']).dtypes.index
numericalfeatures

In [ ]:
categoricalfeatures = data_train.select_dtypes(include = ['object']).dtypes.index
categoricalfeatures

In [ ]:
def UniVA_Cat(data, category):
  plt.figure(figsize = (10,6))
  sns.countplot(x=category, data = data)
  plt.xlabel(category,fontsize = 16, fontweight = 'bold')
  plt.ylabel('Count',fontsize = 16, fontweight = 'bold')
  plt.title('Value counts: \n{}'.format(data_train[category].value_counts(normalize = True)))

  # Rotating xticklabels
  if len(data[category].value_counts()) > 7:
    X = plt.gca().xaxis
    for item in X.get_ticklabels():
      item.set_rotation(90)
  plt.show()


In [ ]:
UniVA_Cat(data_train,'Item_Fat_Content')

In [ ]:
tlfat = 0.597090 + 0.037076 + 0.013141
tlfat

In [ ]:
UniVA_Cat(data_train, 'Item_Type')

In [ ]:
UniVA_Cat(data_train, 'Outlet_Identifier')

In [ ]:
UniVA_Cat(data_train,'Outlet_Size')

In [ ]:
UniVA_Cat(data_train, 'Outlet_Location_Type')

In [ ]:
UniVA_Cat(data_train, 'Outlet_Type')

In [ ]:
data_train['source'] = 'Train'
data_test['source'] = 'Test'
df=pd.concat([data_train,data_test], ignore_index=True)
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
plt.figure(figsize = (10,6))
sns.heatmap(df.isnull(), yticklabels=False,cbar = False,cmap ='Blues')


In [ ]:
def Percentageofmissing():
  miss_item_weight = (df['Item_Weight'].isnull().sum()/len(df))*100
  miss_Outlet_Size = (df['Outlet_Size'].isnull().sum()/len(df))*100

  print('Percentage of missing values in Item_Weight: ' + str(miss_item_weight),"%")
  print('Percentage of missing values in Outlet_Size: ' +str(miss_Outlet_Size),"%")

Percentageofmissing()

In [ ]:
df['Item_Weight'].fillna(df['Item_Weight'].mean(),inplace=True)
df['Outlet_Size'].value_counts()

In [ ]:
df['Outlet_Size'].fillna('Medium', inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.Item_Visibility.value_counts

In [ ]:
df['Item_Visibility'].replace(0.0,value=np.nan,inplace=True)

In [ ]:
df['Item_Visibility']=df['Item_Visibility'].fillna(df.groupby('Item_Identifier')['Item_Visibility'].transform('mean'))

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(x='Item_Fat_Content',data=df)

In [ ]:
df['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'},inplace=True)

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(x='Item_Fat_Content',data=df)

In [ ]:
# Store data for future prediction
test_predictions = df.loc[df['source'] == 'Test']
test_predictions.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)
test_predictions.head()

In [ ]:
category = ['Item_Fat_Content', 'Item_Type', 'Outlet_Size', 'Outlet_Location_Type', 'Outlet_Type']
le = LabelEncoder()
df[category] = df[category].apply(le.fit_transform)

In [ ]:
df.head()

In [ ]:
df.drop(['Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
train = df.loc[df['source'] == 'Train']
test = df.loc[df['source'] == 'Test']

In [ ]:
train.drop(['source'],axis=1,inplace=True)
test.drop(['Item_Outlet_Sales','source'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
x = train.drop(columns="Item_Outlet_Sales")
y = train["Item_Outlet_Sales"]
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2, random_state = 0)

In [ ]:
modelcompare = {}


In [ ]:
l = LinearRegression()

l.fit(X_train,y_train)



In [ ]:
y_predictions = l.predict(X_test)
y_predictions

In [ ]:
modelcompare['Linear Regression'] = [l.score(X_train,y_train)*100,r2_score(y_test,y_predictions)*100]

print("Linear Regression\n\nAccuracy: {}%".format(round(l.score(X_train,y_train)*100)))
print(" score: {}%".format(round(r2_score(y_test,y_predictions)*100)))

In [ ]:
rf = RandomForestRegressor(n_estimators=400,max_depth=6,min_samples_leaf=100,n_jobs=4)
rf.fit(X_train,y_train)

y_predictions = rf.predict(X_test)
y_predictions

In [ ]:
modelcompare['Random Forest'] = [rf.score(X_train,y_train)*100,r2_score(y_test,y_predictions)*100]

print("Random Forest\n\nAccuracy: {}%".format(round(rf.score(X_train,y_train)*100)))
print("score: {}%".format(round(r2_score(y_test,y_predictions)*100)))

In [ ]:
xgb = XGBRegressor(n_estimators = 100, learning_rate=0.05)
xgb.fit(X_train, y_train)
y_predictions = xgb.predict(X_test)
y_predictions

In [ ]:
modelcompare['XGBoost Regressor'] = [xgb.score(X_train,y_train)*100,r2_score(y_test,y_predictions)*100]

print("XGBoost Regressor\n\nAccuracy: {}%".format(round(xgb.score(X_train,y_train)*100)))
print("score: {}%".format(round(r2_score(y_test,y_predictions)*100)))

In [ ]:
modeldf = pd.DataFrame.from_dict(modelcompare).T
modeldf.columns = ['Accuracy', "r2_score"]
modeldf = modeldf.sort_values('Accuracy', ascending=True)
modeldf.style.background_gradient(cmap='PuBu')

In [ ]:
figure = go.Figure(data=[
    go.Bar(name='r2_score', y=modeldf.index, x=modeldf['r2_score'], orientation='h', marker_color='#B6D0E2'),
    go.Bar(name='Accuracy', y=modeldf.index, x=modeldf['Accuracy'], orientation='h', marker_color='#4169e1')
])
figure.update_layout(barmode='group')
figure.show()

In [ ]:
test.head()

In [ ]:
predict = xgb.predict(test)
predict

In [ ]:
test_predictions.head()

In [ ]:
test_predictions["Predicted_Item_Outlet_Sale"] = predict
test_predictions

In [ ]:
test_predictions.to_csv("ResultSubmit.csv",index=False)

In [ ]:
ls=[]
for i in test.columns:
    s = float(input(f"Enter the {i}:"))
    ls.append(s)

In [ ]:
xgb.predict(np.array(ls).reshape(1,-1))

In [ ]:
ls2=[]
for i in test.columns:
    s = float(input(f"Enter the {i}:"))
    ls2.append(s)

In [ ]:
xgb.predict(np.array(ls2).reshape(1,-1))

In [ ]:
joblib.dump(xgb,"Market_model.sav")